In [6]:
import torch
import torchvision
from torchvision.models import ResNet50_Weights, resnet50
from PIL import Image
torch.__version__

'2.0.1'

In [7]:
weights = ResNet50_Weights.DEFAULT
m = resnet50(weights=weights)
m = m.eval()

In [8]:
image = Image.open("cat.jpg")
preprocess = weights.transforms()
data = preprocess(image)
data = data.unsqueeze(0)

In [9]:
data.shape
res = m(data)
id = res.argmax(axis=-1)
print(id)

tensor([282])


In [10]:
import time
import numpy as np
from tqdm import tqdm

import torch._dynamo
torch._dynamo.reset()

#mode = None # 58, 55
#mode = "default" # 56, 55
#mode = "max-autotune" # 57, 55
mode = "reduce-overhead" # 58, 55
print("Compiling...")
m_opt = torch.compile(m, mode=mode)
#m_opt = torch.compile(m, backend='torchxla_trace_once')
res = m_opt(data)
print("Compilation Done")

for model, desc in [(m, "resnet50"), (m_opt, "resnet50_inductor_reduce-overhead")]:
    N = 10; i = 0
    while i < N:
        res = model(data)
        i += 1

    N = 50; i = 0
    TT = []
    while i < N:
        t0 = time.time()
        res = model(data)
        dur = (time.time() - t0) * 1000
        TT.append(dur)
        i += 1

    print(f"{desc},{np.mean(TT):.3f},{np.percentile(TT, 50):.3f}")

Compiling...
Compilation Done
resnet50,63.738,63.093
resnet50_inductor_reduce-overhead,66.830,65.491


In [14]:
import time
import numpy as np
from tqdm import tqdm

import torch._dynamo
torch._dynamo.reset()

def evaluate(mod, inp):
    return mod(inp)

print("Compiling...")
#mode = None # 58, 55
#mode = "default" # 56, 55
#mode = "max-autotune" # 57, 55
mode = "reduce-overhead" # 58, 55
evaluate_opt = torch.compile(evaluate, mode=mode)
res = evaluate_opt(m, data)
print("Compilation Done")

for fun, desc in [(evaluate, "resnet50"), (evaluate_opt, "resnet50_inductor_reduce-overhead")]:
    N = 10; i = 0
    while i < N:
        res = fun(m, data)
        i += 1

    N = 50; i = 0
    TT = []
    while i < N:
        t0 = time.time()
        res = fun(m, data)
        dur = (time.time() - t0) * 1000
        TT.append(dur)
        i += 1

    print(f"{desc},{np.mean(TT):.3f},{np.percentile(TT, 50):.3f}")

Compiling...
Compilation Done
resnet50,67.631,67.993
resnet50_inductor_reduce-overhead,61.248,59.760
